# Making Emoji Predictions for Sentences

In [37]:
!pip install emoji

In [38]:
import emoji

In [39]:
emoji.EMOJI_UNICODE
## Gives the dict of expression->emoji in the emoji dictionary

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_Is

In [40]:
# We used only these 5 emojis as our test data had only these emojis 
# (check the train and test csv files)

emoji_dictionary = {
    "0": "\u2764\uFE0F",
    "1": ":baseball:",
    "2": ":grinning_face_with_big_eyes:",
    "3": ":disappointed_face:",
    "4": ":fork_and_knife:"
}

In [41]:
# Checking out the emojis we have
for ix in emoji_dictionary.values():
    print(emoji.emojize(ix))

❤️
⚾
😃
😞
🍴


## Processing the dataset

In [42]:
import numpy as np
import pandas as pd

In [43]:
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [44]:
train.head() # only the first two columns are useful for us

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [45]:
data = train.values # Converting a CSV file into numpy array
# we have 132 sentences in train dataset and an emoji label corresponding to each
print(data.shape)

(132, 4)


In [46]:
# visualizing the training dataset
for ix in range (5):
    print (data[ix][0], emoji.emojize(emoji_dictionary[str(data[ix][1])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [47]:
X_train_temp = train[0]
y_train = train[1]

X_test_temp = test[0]
y_test = test[1]

In [48]:
# Converting each sentence into a list of words
# (As we have the vector encodings for each word)
X_train = []
X_test = []
for ix in range(X_train_temp.shape[0]):
    X_train.append (X_train_temp[ix].split())

for ix in range(X_test_temp.shape[0]):
    X_test.append (X_test_temp[ix].split())

X_train = np.array (X_train)
X_test = np.array (X_test)

print(X_train.shape,X_test.shape)
print(X_test[1])

(132,) (56,)
['he', 'did', 'not', 'answer']


In [49]:
# Visualizing the Training dataset
for i in range(5):
    print(X_train[i], emoji.emojize(emoji_dictionary[str(y_train[i])]))

['never', 'talk', 'to', 'me', 'again'] 😞
['I', 'am', 'proud', 'of', 'your', 'achievements'] 😃
['It', 'is', 'the', 'worst', 'day', 'in', 'my', 'life'] 😞
['Miss', 'you', 'so', 'much'] ❤️
['food', 'is', 'life'] 🍴


## Converting sentences into embeddings

In [50]:
# Using the Glove vector encodings developed by Stanford
f = open('glove.6B.50d.txt',encoding='utf-8')

In [51]:
## checking out the format
for line in f:
    print(line)
    break

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581



In [52]:
# dictionary embedding_index will have all the
# words mapped to their corresponding embedding vectors
embedding_index = {}
for line in f:
    values = line.split()
    word = values[0]
    emb = np.asarray(values[1:],dtype='float')
    embedding_index[word] = emb
f.close()

In [53]:
# Checking out the dimension of all the word embeddings
emb_dim = embedding_index['eat'].shape[0]
print(emb_dim)

50


## Converting sentences into vectors 
**(Creating the embedding layer output)**

In [54]:
def embedding_output(X):
    maxLen = 10 # no sentence have more than 10 words
    embedding_out = np.zeros((X.shape[0], maxLen, emb_dim)) 
    
    for ix in range(X.shape[0]):
        for ij in range(len(X[ix])):
            # checking if we have the current word in the dict: embedding_index
            try:
                embedding_out[ix][ij] = embedding_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
    return embedding_out

In [55]:
embedding_matrix_train = embedding_output(X_train)
embedding_matrix_test = embedding_output(X_test)

In [56]:
print(embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


## Define the LSTM/RNN model

In [57]:
from keras.layers import *
from keras.models import Sequential
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [58]:
# Converting the output format into one-hot encodings
YT = np_utils.to_categorical(y_train)
Yt = np_utils.to_categorical(y_test)

In [59]:
YT.shape

(132, 5)

In [60]:
# Using a stacked LSTM Model
# Did this first with a normal LSTM model, then tried
# with stacked LSTM to improve accuracy
model = Sequential()
model.add(LSTM(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

# Note: Didn't add an embedding layer as it would have increased
# the number of parameters a lot
# this would have caused overfitting as input data is very small

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [86]:
# Using early stop as my test data was basically overfitting which was 
# visible from the accuracy which almost reached 100% after some epochs
earlystop = EarlyStopping(monitor='val_acc',patience=15)
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=True, save_best_only=True)
hist = model.fit(embedding_matrix_train,YT,epochs=100,batch_size=64,validation_split=0.20, shuffle=True, callbacks=[checkpoint, earlystop])

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 0s 644us/step - loss: 0.0317 - acc: 1.0000 - val_loss: 1.4058 - val_acc: 0.6296

Epoch 00001: val_loss improved from inf to 1.40583, saving model to best_model.h5
Epoch 2/100
105/105 [==============================] - 0s 690us/step - loss: 0.0364 - acc: 0.9905 - val_loss: 1.5339 - val_acc: 0.5926

Epoch 00002: val_loss did not improve from 1.40583
Epoch 3/100
105/105 [==============================] - 0s 677us/step - loss: 0.0290 - acc: 1.0000 - val_loss: 1.7853 - val_acc: 0.5926

Epoch 00003: val_loss did not improve from 1.40583
Epoch 4/100
105/105 [==============================] - 0s 610us/step - loss: 0.0343 - acc: 0.9905 - val_loss: 1.9494 - val_acc: 0.5556

Epoch 00004: val_loss did not improve from 1.40583
Epoch 5/100
105/105 [==============================] - 0s 562us/step - loss: 0.0242 - acc: 1.0000 - val_loss: 2.0220 - val_acc: 0.5556

Epoch 00005: val_loss did not improve fr

In [87]:
# Loading the best model
model.load_weights("best_model.h5")

In [88]:
model.evaluate(embedding_matrix_test, Yt)

56/56 [==============================] - 0s 355us/step


[1.6176965577261788, 0.5535714200564793]

In [89]:
# predict_classes function gives us the output predictions
# in general form (not one hot encoded)
pred = model.predict_classes(embedding_matrix_test)

## Getting Emoji Output

In [90]:
for ix in range(30):
    print(' '.join(X_test[ix]), emoji.emojize(emoji_dictionary[str(pred[ix])]), emoji.emojize(emoji_dictionary[str(y_test[ix])]))

I want to eat 🍴 🍴
he did not answer 😞 😞
he got a raise 😞 😃
she got me a present 😃 ❤️
ha ha ha it was so funny 😃 😃
he is a good friend 😃 ❤️
I am upset ⚾ ❤️
We had such a lovely dinner tonight 😃 ❤️
where is the food 🍴 🍴
Stop making this joke ha ha ha 😃 😃
where is the ball ⚾ ⚾
work is hard 😞 😞
This girl is messing with me ❤️ 😞
are you serious ha ha 😃 😃
Let us go play baseball ⚾ ⚾
This stupid grader is not working 😞 😞
work is horrible 😃 😞
Congratulation for having a baby 😃 😃
stop messing around 😞 😞
any suggestions for dinner 😃 🍴
I love taking breaks 😞 ❤️
you brighten my day ❤️ 😃
I boiled rice 🍴 🍴
she is a bully ❤️ 😞
Why are you feeling bad 😞 😞
I am upset ⚾ 😞
I worked during my birthday 😃 😞
My grandmother is the love of my life ❤️ ❤️
enjoy your break ⚾ 😃
valentine day is near 😃 ❤️
